In [1]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 4.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 3.5 MB 4.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 2.5 MB 4.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 750 kB 4.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/c

In [2]:
import torch
import torch.nn
import torch.nn.functional as F

import numpy
import matplotlib.pyplot as plt

In [3]:
import torch_geometric 
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, VGAE, GAE
from torch_geometric.utils import train_test_split_edges 
from torch_geometric.transforms import RandomLinkSplit

In [15]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)

Processing...
Done!
/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [16]:
# x -> Feature matrix [nodes, number_of_node_features]

In [17]:
data
#edge index = adjacency matrix, test_neg_edges -> edges in test set that are not in the graph, test_pos_edge_index -> edges in test set that are in graph, x -> feature matrix, y -> labels

Data(x=[3327, 3703], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

###Define the encoder

In [10]:
from torch_geometric.nn import VGAE

In [18]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [19]:
#Parameters

out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)


#Initialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [20]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [21]:
from torch.utils.tensorboard import SummaryWriter

In [22]:
writer = SummaryWriter('runs/VGAE_experiment_'+'2d_100_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6098, AP: 0.6356
Epoch: 002, AUC: 0.6217, AP: 0.6468
Epoch: 003, AUC: 0.6251, AP: 0.6500
Epoch: 004, AUC: 0.6268, AP: 0.6519
Epoch: 005, AUC: 0.6273, AP: 0.6524
Epoch: 006, AUC: 0.6275, AP: 0.6528
Epoch: 007, AUC: 0.6276, AP: 0.6528
Epoch: 008, AUC: 0.6276, AP: 0.6528
Epoch: 009, AUC: 0.6275, AP: 0.6528
Epoch: 010, AUC: 0.6274, AP: 0.6524
Epoch: 011, AUC: 0.6275, AP: 0.6525
Epoch: 012, AUC: 0.6270, AP: 0.6521
Epoch: 013, AUC: 0.6267, AP: 0.6516
Epoch: 014, AUC: 0.6264, AP: 0.6511
Epoch: 015, AUC: 0.6259, AP: 0.6507
Epoch: 016, AUC: 0.6253, AP: 0.6499
Epoch: 017, AUC: 0.6249, AP: 0.6493
Epoch: 018, AUC: 0.6246, AP: 0.6488
Epoch: 019, AUC: 0.6239, AP: 0.6480
Epoch: 020, AUC: 0.6229, AP: 0.6468
Epoch: 021, AUC: 0.6214, AP: 0.6456
Epoch: 022, AUC: 0.6201, AP: 0.6445
Epoch: 023, AUC: 0.6188, AP: 0.6435
Epoch: 024, AUC: 0.6178, AP: 0.6428
Epoch: 025, AUC: 0.6168, AP: 0.6416
Epoch: 026, AUC: 0.6163, AP: 0.6411
Epoch: 027, AUC: 0.6156, AP: 0.6404
Epoch: 028, AUC: 0.6153, AP: